In [1]:
import findspark
import numpy as np
import pandas as pd
import os
import re
findspark.init()
findspark.find()

'D:\\Programs\\spark-3.5.0-bin-hadoop3'

In [2]:
import pyspark
sc = pyspark.SparkContext("local", "lab2")

In [3]:
lines = sc.textFile("2.txt")
links = lines.map(lambda line: line.split()).map(lambda pair: (pair[0], pair[1]))

In [4]:
pages = links.flatMap(lambda x: x).distinct()
page_count = pages.count()
ranks = pages.map(lambda x: (x, 1.0))
links = links.groupByKey().mapValues(list).cache()
links.collect()

[('0', ['1', '30', '87']),
 ('2', ['0', '7', '10']),
 ('3', ['0']),
 ('4', ['0']),
 ('5', ['0', '29', '93']),
 ('6', ['0', '1', '2', '4', '5', '16', '70', '86']),
 ('8', ['0', '1', '2', '3', '7', '48']),
 ('9', ['0', '4', '5', '7', '8', '10', '14', '32', '54']),
 ('13', ['0', '1', '2', '6', '9', '66']),
 ('15', ['0', '2', '3', '7', '13', '45']),
 ('22', ['0', '3', '6', '8', '13', '25', '60', '79']),
 ('23', ['0', '10', '14', '19', '22']),
 ('24', ['0', '4', '10', '17', '18']),
 ('27', ['0', '1', '4', '17', '21']),
 ('30', ['3', '5', '7', '23']),
 ('32', ['0', '3', '11', '17', '80', '87']),
 ('35', ['0', '7', '9', '22']),
 ('38', ['0', '3', '4', '7', '9']),
 ('39', ['0', '3', '5']),
 ('41', ['0', '3', '9', '18']),
 ('46', ['0', '3', '13', '32', '33', '70', '88']),
 ('59', ['0', '7', '18', '41']),
 ('70', ['0', '27', '42']),
 ('81', ['0', '7', '12', '15', '42']),
 ('87', ['49', '64']),
 ('91', ['0', '6', '33', '79']),
 ('7', ['1', '4', '5', '6', '21', '50', '65']),
 ('12', ['1', '6', '7'

In [5]:
def computeContribs(node_urls_rank):
    _, (urls, rank) = node_urls_rank
    nb_urls = len(urls)
    for url in urls:
        yield url, rank / nb_urls

In [6]:
from operator import add
import time
start_time = time.time()

for iteration in range(0, 15):
    contribs = links.join(ranks).flatMap(computeContribs)
    contribs = links.fullOuterJoin(contribs).mapValues(lambda x : x[1] or 0.0)
    ranks = contribs.reduceByKey(add)
    ranks = ranks.mapValues(lambda rank: rank * 0.85 + 0.15)
    
total_sum = ranks.values().sum()
result = ranks.mapValues(lambda x: x / total_sum).collectAsMap()

for key, value in sorted(result.items(), key=lambda x: x[1], reverse=True):
    print(f'{key}: {round(value*100, 3)}%')
    
end_time = time.time()
print(f"Program execution time: {end_time - start_time} seconds")

0: 10.06%
7: 6.105%
1: 4.858%
4: 3.356%
30: 3.236%
6: 3.178%
87: 3.175%
3: 3.128%
2: 3.083%
5: 2.829%
14: 2.499%
9: 2.343%
10: 2.249%
49: 1.704%
64: 1.686%
29: 1.615%
18: 1.506%
13: 1.478%
21: 1.444%
25: 1.259%
15: 1.226%
12: 1.202%
16: 1.162%
44: 1.136%
11: 1.108%
50: 1.087%
17: 1.068%
93: 1.027%
23: 1.007%
8: 0.989%
22: 0.983%
38: 0.976%
65: 0.976%
24: 0.747%
34: 0.722%
28: 0.718%
41: 0.71%
45: 0.694%
27: 0.652%
48: 0.621%
33: 0.62%
31: 0.604%
39: 0.599%
53: 0.573%
70: 0.569%
86: 0.528%
59: 0.518%
62: 0.513%
61: 0.504%
42: 0.498%
83: 0.493%
32: 0.486%
85: 0.464%
47: 0.438%
19: 0.424%
54: 0.411%
71: 0.41%
66: 0.399%
35: 0.399%
79: 0.398%
57: 0.398%
68: 0.396%
89: 0.382%
67: 0.375%
94: 0.354%
46: 0.342%
91: 0.341%
74: 0.335%
26: 0.327%
77: 0.323%
55: 0.313%
40: 0.302%
84: 0.296%
80: 0.294%
82: 0.294%
60: 0.293%
76: 0.292%
58: 0.281%
36: 0.26%
75: 0.252%
97: 0.247%
92: 0.236%
95: 0.235%
88: 0.23%
52: 0.229%
37: 0.221%
99: 0.221%
96: 0.216%
56: 0.189%
63: 0.189%
51: 0.189%
72: 0.189%
43:

In [12]:
links_dict = links.collectAsMap()

import networkx as nx
g = nx.DiGraph(links_dict)
res_nx = nx.pagerank(g, alpha=0.85)
for key, value in sorted(res_nx.items(), key=lambda x: x[1], reverse=True):
    print(f'{key}: {round(value*100, 3)}%')

0: 10.026%
7: 6.094%
1: 4.835%
4: 3.349%
30: 3.219%
6: 3.174%
87: 3.156%
3: 3.129%
2: 3.077%
5: 2.818%
14: 2.5%
9: 2.345%
10: 2.246%
49: 1.694%
64: 1.674%
29: 1.614%
18: 1.506%
13: 1.48%
21: 1.443%
25: 1.255%
15: 1.231%
12: 1.207%
16: 1.164%
44: 1.13%
11: 1.11%
50: 1.087%
17: 1.07%
93: 1.023%
23: 1.003%
8: 0.992%
22: 0.985%
38: 0.976%
65: 0.974%
24: 0.748%
28: 0.719%
34: 0.719%
41: 0.711%
45: 0.698%
27: 0.656%
33: 0.622%
48: 0.622%
31: 0.605%
39: 0.6%
53: 0.573%
70: 0.57%
86: 0.528%
59: 0.52%
62: 0.513%
61: 0.506%
42: 0.501%
83: 0.495%
32: 0.488%
85: 0.465%
47: 0.442%
19: 0.426%
71: 0.413%
54: 0.413%
35: 0.401%
79: 0.401%
66: 0.401%
57: 0.401%
68: 0.398%
89: 0.385%
67: 0.378%
94: 0.356%
46: 0.346%
91: 0.343%
74: 0.338%
26: 0.33%
77: 0.324%
55: 0.316%
40: 0.305%
84: 0.299%
80: 0.297%
82: 0.296%
60: 0.296%
76: 0.294%
58: 0.284%
36: 0.263%
75: 0.255%
97: 0.25%
92: 0.239%
95: 0.237%
88: 0.233%
52: 0.231%
99: 0.224%
37: 0.224%
96: 0.218%
81: 0.191%
73: 0.191%
78: 0.191%
43: 0.191%
72: 0.191

In [8]:
print(pyspark.__version__)

3.5.0
